In [39]:
# Read csv to Dictionary
import csv
reader = csv.reader(open('unit_conversions.csv', 'r'), delimiter=';')
dict_from_csv = {} # start with empty dictionary

####
## Add code to check whether two columns or not
## Add code for input types (string, float)
####

for k, v in reader:
    dict_from_csv[k] = v # add key and value to dict

print(dict_from_csv) # test output

{'distance_SI_km': '0,001', 'distance_SI_cm': '100', 'distance_SI_mm': '1000'}


In [10]:
# Convert Units
########
# Self-made Dictionary

# All conversions are made to SI units
# https://en.wikipedia.org/wiki/United_States_customary_units#Units_of_length

dict_unit = {
    # distance to Meter (SI)
    'distance_SI_km':0.001, # kilometer
    'distance_SI_cm':100, # centimeter
    'distance_SI_mm':1000, # millimeter
    'distance_USCS_mi.':1609.344, # miles
    'distance_USCS_ft':0.3048, # feet
    'distance_USCS_in':0.0254, # inch
    # volume to Liter (SI)
    'volume_USCS_cu_in':0.016387064, # cubic inch
    'volume_USCS_cu_ft':28.316846592, # cubic feet
    'volume_USCS_cu_yd':764.554857984, # cubic yard
    'volume_USCS_bbl':158.987294928, # oil barrel
}

In [ ]:
# Make universal dictionary reader for unit conversions

def retrieve_conv_unit_from_dict(dict, unit_subject, unit_system, unit_specs):
    retrieval_value = unit_subject + '_' + unit_system + '_' + unit_specs
    return(dict.get(retrieval_value, 'ERROR'))

In [12]:
# Convert Units
########
# PintPy: https://pint.readthedocs.io/en/0.11/

## Error: currently this does not want to work

import pint
ureg = pint.UnitRegistry() 

'''
# Examplary code how to use pintpy: 
print(3 * ureg.meter + 4 * ureg.cm)
'''

def convert_pintpy(data_in, unit_in, unit_out):
    ureg = pint.UnitRegistry()

    unit_in = "ureg." + unit_in # this string conversion seems to create error
    unit_out = "ureg." + unit_out # but how to create this variable otherwise?
    out = (1 * unit_out + data_in * unit_in)
    out = out - (1 * unit_out)

    return(out)

In [13]:
# Convert Units
########
# PyPi: Unit-Converter

## Error: currently this does not want to work
from unit_convert import UnitConvert

'''
# Examplary code how to use pypi:
# Yards + kilometres to miles
x = UnitConvert(yards=136.23, kilometres=60).miles
print(x)
# Bytes to terabytes
x = UnitConvert(b=19849347813875).tb
print(x)
'''

def convert_unit_converter(data_in, unit_in, unit_out):
    from unit_convert import UnitConvert
    return(UnitConvert(unit_in=data_in).unit_out)

37.35976780046479
18.052876670367368


In [14]:
# Convert Time
########
#   PyPi: Workdays

## Needs to be made

In [15]:
# Convert Time
########
#   PyPi: BusinessHours

## Needs to be made

In [16]:
# General importing
import pandas as pd
import numpy as np

from datetime import datetime
start_time = datetime.now() # for time loop

# START TIME LOOP #
####### import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=0, sep=';',low_memory=False)
df_import_supplier_A.head()

## Here more code needs to come
'''
#######
#   PintPy
# convert_pintpy(1,"meter","cm") # does not work
#   PyPi: Unit-converter
# #convert_unit_converter(100, kilometres, miles) # does not work
#   PyPi: Workdays
#   PyPi: BusinessHours
#######
'''

# END OF TIME LOOP #
time_elapsed = datetime.now() - start_time
print('Time elapsed (h:m:s.ms) {}'.format(time_elapsed))

print(df_import_supplier_A.columns)

Time elapsed (h:m:s.ms) 0:00:00.470738
Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Temperatures', 'Temperatures.1', 'Mass', 'Mass.1',
       'Distance', 'Distance.1', 'Distance.2', 'Distance.3'],
      dtype='object')


In [41]:
## Trying out stuff

y = df_import_supplier_A['Temperatures']
y.iloc[2:y.size]
#y.apply(UnitConvert(yards=136.23, kilometres=60).miles)
#y.apply(UnitConvert, args=())

#y.apply(retrieve_conv_unit_from_dict(dict_from_csv,'distance','SI', 'mm'))

#y.apply(retrieve_conv_unit_from_dict('distance','SI', 'mm'))
#y.apply(UnitConvert(unit_in=data_in).unit_out)

2          0
3         80
4          C
5         68
6         58
          ..
199994     5
199995    72
199996     5
199997    26
199998     5
Name: Temperatures, Length: 199997, dtype: object

In [ ]:
'''
Reserved space to use like notepad, handy to retrieve useful code
'''
# drop rows
#df_import_supplier_A.drop([1,5])

# drop column
#df_import_supplier_A.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], axis=1)

## Perform library search, handmade python

## Run large table
